# A2

In [ ]:
import math
from itertools import count

import numpy as np
import matplotlib.pyplot as plt
from seaborn.external.docscrape import header

In [543]:
with open('vidu4_lin_reg.txt') as f:
    lines = f.readlines()

x_data = []
ntm = []
y_data = []
lines.pop(0)

for line in lines:
    splitted = line.replace('\n', '').split(' ')
    splitted.pop(0)
    splitted = list(map(float, splitted))
    x_data.append(splitted[:5])
    ntm.append(splitted[5])

x_data = np.asarray(x_data)
y_data = np.zeros(len(ntm))
for i in range(len(y_data)):
    if ntm[i] >= 1:
        y_data[i] = 1
    else:
        y_data[i] = 0
y_data = np.asarray(y_data)
print(y_data)

[1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 1.]


In [544]:
k = 6
def distance(array, value):
    array = np.array(array)
    return np.linalg.norm(array - value,ord=2, axis=1)

def find_nearest_index(array, value, k):
    array_D = distance(array, value)
    return np.argsort(array_D)[:k]

data_len = len(x_data)

X_train = np.array(x_data[:80])
Y_train = np.array(y_data[:80])
X_test = np.array(x_data[80:data_len])
Y_test = np.array(y_data[80:data_len])

In [545]:
Y_pred = np.zeros(len(X_test))
for i in range(len(X_test)):
    indexis = find_nearest_index(X_train, X_test[i], k)
    for id in indexis:
        Y_pred[i] = Y_pred[i] + Y_train[id]
    Y_pred[i] = Y_pred[i]/len(indexis)
    if Y_pred[i] >= 0.5:
        Y_pred[i] = 1
    else:
        Y_pred[i] = 0
    print(Y_pred[i], ' | ', Y_test[i])

0.0  |  0.0
1.0  |  0.0
1.0  |  1.0
1.0  |  0.0
1.0  |  0.0
0.0  |  0.0
1.0  |  1.0
1.0  |  0.0
1.0  |  1.0
0.0  |  0.0
1.0  |  0.0
0.0  |  0.0
1.0  |  1.0
0.0  |  1.0
0.0  |  1.0
0.0  |  0.0
1.0  |  1.0
1.0  |  1.0
1.0  |  0.0
0.0  |  1.0


In [547]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
Y_pred = Y_pred.astype(int)
Y_test = Y_test.astype(int)

acc = accuracy_score(Y_test, Y_pred)
prec = precision_score(Y_test, Y_pred)
rec = recall_score(Y_test, Y_pred)
cm = confusion_matrix(Y_test, Y_pred)

print("\n=== Evaluation Metrics ===")
print(f"Accuracy : {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall   : {rec:.2f}")



=== Evaluation Metrics ===
Accuracy : 0.55
Precision: 0.50
Recall   : 0.67


# Bài 1

## Logistic Regression

In [548]:
 import numpy as np
 import pandas as pd

In [549]:
data = pd.read_csv("Admission_Predict.csv")
sn = data['Serial No.'].tolist()

gre = data['GRE Score'].tolist()
X1 = np.asarray(gre)

tfl = data['TOEFL Score'].tolist()
X2 = np.asarray(tfl)

unirt = data['University Rating'].tolist()
X3 = np.asarray(unirt)

sop = data['SOP'].tolist()
X4 = np.asarray(sop)

lor1 = data['LOR '].tolist()
X5 = np.asarray(lor1)

cgpa1 = data['CGPA'].tolist()
X6 = np.asarray(cgpa1)

research_exp = data['Research'].tolist()
X7 = np.asarray(research_exp)

prob_Admit = data['Chance of Admit'].tolist()
Yt = np.asarray(prob_Admit)



In [550]:
def sigmoid(s):
    s = np.clip(s, -500, 500)
    return 1/(1 + np.exp(-s))

def logistic_sigmoid_regression(X, y, w_init, eta, tol = 1e-4, max_count = 10000):
    # method to calculate model logistic regression by Stochastic Gradient Descent method
    # eta: learning rate; tol: tolerance; max_count: maximum iterates
    w = [w_init]
    it = 0
    N = X.shape[1]
    d = X.shape[0]
    count = 0
    check_w_after = 20
    # loop of stochastic gradient descent
    while count < max_count:
    # shuffle the order of data (for stochastic gradient descent).
    # and put into mix_id
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = y[i]
            zi = sigmoid(np.dot(w[-1].T, xi))
            w_new = w[-1] + eta*(yi - zi)*xi
            count += 1
            # stopping criteria
            if count%check_w_after == 0:
                if np.linalg.norm(w_new - w[-check_w_after]) < tol:
                    return w
            w.append(w_new)
    return w

In [551]:
Y = np.array([1 if p >= 0.75 else 0 for p in Yt])
X = np.stack((X1, X2, X3, X4, X5, X6, X7), axis=0)
Xbar = np.concatenate((np.ones((1, X.shape[1])), X), axis=0).T

In [552]:
X_train = Xbar[:350,:]
Y_train = Y[:350]
X_test = Xbar[350:,:]
Y_test = Y[350:]

In [559]:
d = X_train.shape[1]
w_init = np.random.randn(d,1)
eta = .05
w = logistic_sigmoid_regression(X_train.T, Y_train, w_init, eta)
print(w[-1])

[[ -5.85496782]
 [-72.18562892]
 [202.68714634]
 [139.69520769]
 [110.05627657]
 [ 90.62185502]
 [ 42.16971288]
 [ 57.95083707]]


In [560]:
y_prod = sigmoid(np.dot(w[-1].T, X_test.T)).flatten()
y_pred = (y_prod >= 0.5).astype(int)
print("Các hệ số của mô hình (w):")
print(w[-1].flatten())
TP = np.sum((y_pred == 1) & (Y_test == 1))
TN = np.sum((y_pred == 0) & (Y_test == 0))
FP = np.sum((y_pred == 1) & (Y_test == 0))
FN = np.sum((y_pred == 0) & (Y_test == 1))

accuracy = (TP + TN) / len(Y_test)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"\nĐộ chính xác (Accuracy): {accuracy:.4f}")
print(f"Độ chính xác dương (Precision): {precision:.4f}")
print(f"Độ nhạy (Recall): {recall:.4f}")
for i in range(len(y_pred)):
    print(f"{y_pred[i]} || {Y_test[i]}")

Các hệ số của mô hình (w):
[ -5.85496782 -72.18562892 202.68714634 139.69520769 110.05627657
  90.62185502  42.16971288  57.95083707]

Độ chính xác (Accuracy): 0.7600
Độ chính xác dương (Precision): 0.7083
Độ nhạy (Recall): 0.7727
1 || 0
1 || 0
0 || 0
1 || 0
0 || 0
0 || 0
1 || 1
1 || 0
0 || 0
0 || 1
1 || 1
1 || 1
1 || 1
0 || 0
0 || 1
1 || 1
0 || 0
0 || 0
0 || 0
0 || 0
0 || 0
1 || 1
1 || 1
1 || 1
0 || 0
0 || 0
0 || 0
1 || 0
0 || 0
0 || 0
0 || 1
0 || 0
1 || 1
1 || 0
1 || 1
1 || 1
0 || 0
1 || 0
0 || 0
1 || 1
0 || 0
0 || 0
1 || 1
0 || 1
1 || 1
1 || 1
0 || 1
1 || 1
0 || 0
1 || 1


## Regression

In [561]:
 import numpy as np
 import pandas as pd
 import math

In [562]:
data = pd.read_csv("Admission_Predict.csv")
sn = data['Serial No.'].tolist()

gre = data['GRE Score'].tolist()
X1 = np.asarray(gre)

tfl = data['TOEFL Score'].tolist()
X2 = np.asarray(tfl)

unirt = data['University Rating'].tolist()
X3 = np.asarray(unirt)

sop = data['SOP'].tolist()
X4 = np.asarray(sop)

lor1 = data['LOR '].tolist()
X5 = np.asarray(lor1)

cgpa1 = data['CGPA'].tolist()
X6 = np.asarray(cgpa1)

research_exp = data['Research'].tolist()
X7 = np.asarray(research_exp)

prob_Admit = data['Chance of Admit'].tolist()
Yt = np.asarray(prob_Admit)



In [563]:
Y = np.asarray(np.array([1 if p >= 0.75 else 0 for p in Yt]))
X = np.asarray(np.stack((X1, X2, X3, X4, X5, X6, X7), axis=0).T)

In [564]:
def qr_householder(A):
    M, N = A.shape
    Q = np.eye(M)
    R = A.copy().astype(float)

    for n in range(N):
        x = R[n:, n]
        ro = -np.sign(x[0]) * np.linalg.norm(x)
        v = x.copy()
        v[0] -= ro
        v /= np.linalg.norm(v)

        R[n:, :] -= 2 * np.outer(v, v @ R[n:, :])
        Q[:, n:] -= 2 * np.outer(Q[:, n:] @ v, v)

    return Q, R

In [565]:
def linear_regression(x_data, y_data):
    X_bar = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)
    Q, R = qr_householder(X_bar)
    N = X_bar.shape[1]
    R1 = R[:N, :]
    Q1 = Q[:, :N]
    w = np.linalg.inv(R1) @ Q1.T @ y_data
    return w

In [566]:
x_train = X[:350,:]
y_train = Y[:350]
x_test = X[350:,:]
y_test = Y[350:]

In [567]:
w = linear_regression(x_train, y_train)
print("\nSo sánh giá trị thực tế và dự đoán (tập test):")
print(f"{'STT':>3} | {'Thực tế (y_test)':>15} | {'Dự đoán (y_pred)':>17} | {'Sai số (error)':>15}")
print("-" * 60)
for i in range(len(y_test)):
    print(f"{i+1:3d} | {y_test[i]:15.0f} | {y_pred[i]:17.0f}")

mse = np.mean((y_pred - y_test) ** 2)
print("Trung bình bình phương sai số (MSE):", mse)
y_class = (y_pred >= 0.5).astype(int)
accuracy = np.mean(y_class == y_test)
print("Độ chính xác:", accuracy)


So sánh giá trị thực tế và dự đoán (tập test):
STT | Thực tế (y_test) |  Dự đoán (y_pred) |  Sai số (error)
------------------------------------------------------------
  1 |               0 |                 1
  2 |               0 |                 1
  3 |               0 |                 0
  4 |               0 |                 1
  5 |               0 |                 0
  6 |               0 |                 0
  7 |               1 |                 1
  8 |               0 |                 1
  9 |               0 |                 0
 10 |               1 |                 0
 11 |               1 |                 1
 12 |               1 |                 1
 13 |               1 |                 1
 14 |               0 |                 0
 15 |               1 |                 0
 16 |               1 |                 1
 17 |               0 |                 0
 18 |               0 |                 0
 19 |               0 |                 0
 20 |               0 |         

## Naive Bayes

In [568]:
import numpy as np
import pandas as pd
import time
from math import sqrt, pi, exp

In [569]:
data = pd.read_csv("Admission_Predict.csv")
X = data[['GRE Score', 'TOEFL Score', 'University Rating',
          'SOP', 'LOR ', 'CGPA', 'Research']].values
Yt = data['Chance of Admit'].values
Y = np.array([1 if p >= 0.75 else 0 for p in Yt])
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [570]:

def sigmoid(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def logistic_sgd(X, y, eta=0.01, tol=1e-4, max_iter=10000):
    n_samples, n_features = X.shape
    w = np.zeros((n_features, 1))
    y = y.reshape(-1, 1)
    prev_w = w.copy()
    for i in range(max_iter):
        idx = np.random.randint(0, n_samples)
        xi = X[idx].reshape(-1, 1)
        yi = y[idx]
        zi = sigmoid(np.dot(w.T, xi))
        w += eta * (yi - zi) * xi
        if i % 500 == 0 and np.linalg.norm(w - prev_w) < tol:
            break
        prev_w = w.copy()
    return w

# Hàm đánh giá
def evaluate(y_true, y_pred):
    TP = np.sum((y_pred == 1) & (y_true == 1))
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))

    accuracy = (TP + TN) / len(y_true)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return accuracy, precision, recall, (TP, TN, FP, FN)

# Thêm cột bias 1
Xbar = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

# Logistic Regression
start_lr = time.time()
w = logistic_sgd(Xbar, Y, eta=0.01)
y_prob = sigmoid(np.dot(Xbar, w))
y_pred_lr = (y_prob >= 0.5).astype(int).flatten()
end_lr = time.time()

acc_lr, pre_lr, rec_lr, conf_lr = evaluate(Y, y_pred_lr)

In [571]:
def gaussian_prob(x, mean, std):
    std = max(std, 1e-6)
    return (1 / (sqrt(2 * pi) * std)) * exp(- ((x - mean) ** 2) / (2 * std ** 2))

def train_gnb(X, y):
    classes = np.unique(y)
    mu_list = []
    std_list = []
    pi_list = []
    for c in classes:
        X_c = X[y == c]
        mu_list.append(X_c.mean(axis=0))
        std_list.append(X_c.std(axis=0))
        pi_list.append(len(X_c) / len(X))
    return np.array(mu_list), np.array(std_list), np.array(pi_list)

def predict_gnb(X, mu_list, std_list, pi_list):
    y_pred = []
    for x in X:
        scores = []
        for mu, std, pi in zip(mu_list, std_list, pi_list):
            likelihood = np.prod([gaussian_prob(xi, mui, si) for xi, mui, si in zip(x, mu, std)])
            scores.append(likelihood * pi)
        y_pred.append(np.argmax(scores))
    return np.array(y_pred)

start_nb = time.time()
mu_list, std_list, pi_list = train_gnb(X, Y)
y_pred_nb = predict_gnb(X, mu_list, std_list, pi_list)
end_nb = time.time()

acc_nb, pre_nb, rec_nb, conf_nb = evaluate(Y, y_pred_nb)

In [572]:
start_nb = time.time()
mu_list, std_list, pi_list = train_gnb(X, Y)
y_pred_nb = predict_gnb(X, mu_list, std_list, pi_list)
end_nb = time.time()

acc_nb, pre_nb, rec_nb, conf_nb = evaluate(Y, y_pred_nb)

print("🔹 Logistic Regression (SGD)")
print(f"Accuracy : {acc_lr:.4f}, Precision: {pre_lr:.4f}, Recall: {rec_lr:.4f}")
print(f"Time     : {end_lr - start_lr:.4f}s")
print(f"Confusion (TP, TN, FP, FN): {conf_lr}\n")

print("🔹 Gaussian Naive Bayes")
print(f"Accuracy : {acc_nb:.4f}, Precision: {pre_nb:.4f}, Recall: {rec_nb:.4f}")
print(f"Time     : {end_nb - start_nb:.4f}s")
print(f"Confusion (TP, TN, FP, FN): {conf_nb}\n")

print("🔸 So sánh tổng kết:")
print("Thuật toán                  Accuracy    Precision     Recall    Time(s)")
print("----------------------------------------------------------------------")
print(f"Logistic Regression (SGD)     {acc_lr:.3f}        {pre_lr:.3f}      {rec_lr:.3f}     {(end_lr - start_lr):.4f}")
print(f"Gaussian Naive Bayes          {acc_nb:.3f}        {pre_nb:.3f}      {rec_nb:.3f}     {(end_nb - start_nb):.4f}")


🔹 Logistic Regression (SGD)
Accuracy : 0.8750, Precision: 0.8736, Recall: 0.8444
Time     : 0.0690s
Confusion (TP, TN, FP, FN): (np.int64(152), np.int64(198), np.int64(22), np.int64(28))

🔹 Gaussian Naive Bayes
Accuracy : 0.8700, Precision: 0.8721, Recall: 0.8333
Time     : 0.0278s
Confusion (TP, TN, FP, FN): (np.int64(150), np.int64(198), np.int64(22), np.int64(30))

🔸 So sánh tổng kết:
Thuật toán                  Accuracy    Precision     Recall    Time(s)
----------------------------------------------------------------------
Logistic Regression (SGD)     0.875        0.874      0.844     0.0690
Gaussian Naive Bayes          0.870        0.872      0.833     0.0278


## Dùng thư viện Scikit-Learn

In [573]:
import numpy as np
import pandas as pd
import time
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

data = pd.read_csv("Admission_Predict.csv")
X = data[['GRE Score','TOEFL Score','University Rating','SOP','LOR ','CGPA','Research']].values
Yt = data['Chance of Admit'].values
Y = np.array([1 if p >= 0.75 else 0 for p in Yt])

X = (X - X.mean(axis=0)) / X.std(axis=0)

start = time.time()
logReg = linear_model.LogisticRegression(penalty='l2', max_iter=10000)
logReg.fit(X, Y)
end = time.time()

Y_pred = logReg.predict(X)

acc = accuracy_score(Y, Y_pred)
prec = precision_score(Y, Y_pred)
rec = recall_score(Y, Y_pred)
cm = confusion_matrix(Y, Y_pred)

print("🔹 Logistic Regression (Scikit-learn)")
print(f"Accuracy : {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")
print(f"Time     : {end - start:.4f}s")
print("Confusion matrix:\n", cm)
print("\nCác hệ số w:")
print(logReg.coef_)
print("Intercept:", logReg.intercept_)


🔹 Logistic Regression (Scikit-learn)
Accuracy : 0.8875, Precision: 0.8902, Recall: 0.8556
Time     : 0.0177s
Confusion matrix:
 [[201  19]
 [ 26 154]]

Các hệ số w:
[[0.57504585 0.53335519 0.38708674 0.4194294  0.36231289 1.85302956
  0.39521135]]
Intercept: [-0.4647888]


In [574]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import time

# ==============================================================
# 🟦 BỘ DỮ LIỆU 1: DỮ LIỆU 1 CHIỀU (X, y)
# ==============================================================
X = np.array([[0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 1.75, 2.00, 2.25, 2.50,
               2.75, 3.00, 3.25, 3.50, 4.00, 4.25, 4.50, 4.75, 5.00, 5.50]]).T
y = np.array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
              1, 0, 1, 0, 1, 1, 1, 1, 1, 1])

start = time.time()
logReg1 = linear_model.LogisticRegression(penalty='l2')
logReg1.fit(X, y)
end = time.time()

y_pred1 = logReg1.predict(X)

print("🔹 [Bộ 1D] Logistic Regression (Scikit-learn)")
print("Coefficients:", logReg1.coef_)
print("Intercept   :", logReg1.intercept_)
print("Accuracy :", accuracy_score(y, y_pred1))
print("Precision:", precision_score(y, y_pred1))
print("Recall   :", recall_score(y, y_pred1))
print("Time     :", end - start)
print("Confusion matrix:\n", confusion_matrix(y, y_pred1))
print("-" * 60)


# ==============================================================
# 🟩 BỘ DỮ LIỆU 2: DỮ LIỆU 2 CHIỀU NGẪU NHIÊN (X1, X2)
# ==============================================================
means = [[2, 2], [4, 2]]
cov = [[.7, 0], [0, .7]]
N = 20

X1 = np.random.multivariate_normal(means[0], cov, N)
X2 = np.random.multivariate_normal(means[1], cov, N)
X_2D = np.vstack((X1, X2))
y_2D = np.array([0]*N + [1]*N)

start = time.time()
logReg2 = linear_model.LogisticRegression(penalty='l2')
logReg2.fit(X_2D, y_2D)
end = time.time()

y_pred2 = logReg2.predict(X_2D)

print("🔹 [Bộ 2D] Logistic Regression (Scikit-learn)")
print("Coefficients:", logReg2.coef_)
print("Intercept   :", logReg2.intercept_)
print("Accuracy :", accuracy_score(y_2D, y_pred2))
print("Precision:", precision_score(y_2D, y_pred2))
print("Recall   :", recall_score(y_2D, y_pred2))
print("Time     :", end - start)
print("Confusion matrix:\n", confusion_matrix(y_2D, y_pred2))


🔹 [Bộ 1D] Logistic Regression (Scikit-learn)
Coefficients: [[1.14881183]]
Intercept   : [-3.14004565]
Accuracy : 0.8
Precision: 0.8
Recall   : 0.8
Time     : 0.0075550079345703125
Confusion matrix:
 [[8 2]
 [2 8]]
------------------------------------------------------------
🔹 [Bộ 2D] Logistic Regression (Scikit-learn)
Coefficients: [[ 2.13771786 -0.0576235 ]]
Intercept   : [-6.1466587]
Accuracy : 0.925
Precision: 0.9473684210526315
Recall   : 0.9
Time     : 0.00414729118347168
Confusion matrix:
 [[19  1]
 [ 2 18]]


# Bài 2

In [575]:
import pandas as pd
import time

In [576]:
data = pd.read_csv("banking.csv")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [577]:
dict_month = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6,
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
data['month'] = data['month'].map(dict_month)

dict_day = {'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5}
data['day_of_week'] = data['day_of_week'].map(dict_day)

# Convert binary fields
for col in ['default', 'housing', 'loan']:
    data[col] = data[col].replace({'no':0, 'yes':1, 'unknown':0}).astype(int)

# Convert pdays: -1 or 999 → 0 (not contacted), else 1
data['pdays'] = data['pdays'].apply(lambda x: 0 if x in [-1, 999] else 1)

# One-hot encoding for categorical variables
categorical_cols = ['job', 'marital', 'education', 'contact', 'poutcome']
for col in categorical_cols:
    dummies = pd.get_dummies(data[col], prefix=col, drop_first=True).astype(int)
    data = pd.concat([data, dummies], axis=1)
    data.drop(col, axis=1, inplace=True)

# Encode target variable
data['y'] = data['y'].replace({'no':0, 'yes':1}).astype(int)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17188\4088146044.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'no':0, 'yes':1, 'unknown':0}).astype(int)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17188\4088146044.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({'no':0, 'yes':1, 'unknown':0}).astype(int)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17188\4088146044.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future 

In [578]:
data = data.dropna()
X = data.drop('y', axis=1).values.T
y = data['y'].values
X = np.vstack([np.ones((1, X.shape[1])), X]).T

In [579]:
def sigmoid(s):
    s = np.clip(s, -500, 500)
    return 1 / (1 + np.exp(-s))
def logistic_sigmoid_regression(X, y, w_init, eta, tol=1e-4, max_count=10000):
    w = [w_init]
    it = 0
    N = X.shape[1]
    d = X.shape[0]
    count = 0
    check_w_after = 20
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = y[i]
            zi = sigmoid(np.dot(w[-1].T, xi))
            w_new = w[-1] + eta * (yi - zi) * xi
            count += 1
            if count % check_w_after == 0:
                if np.linalg.norm(w_new - w[-check_w_after]) < tol:
                    return w
            w.append(w_new)
    return w

In [580]:
train_size = int(0.8 * len(X))
X_train = X[:train_size,:]
Y_train =y[:train_size]
X_test = X[train_size:,:]
Y_test = y[train_size:]
print(X_train.shape)
print(X_test.shape)

(32950, 40)
(8238, 40)


In [586]:
d = X_train.shape[1]
w_init = np.random.randn(d, 1)
eta = 0.01
w = logistic_sigmoid_regression(X_train.T, Y_train, w_init, eta)
print(w[-1].shape)

(40, 1)


In [587]:
start_log = time.time()
y_prod = sigmoid(np.dot(w[-1].T, X_test.T))
y_pred = (y_prod >= 0.5).astype(int)
end_log = time.time()
print("Các hệ số của mô hình (w):")
print(w[-1].flatten())
TP = np.sum((y_pred == 1) & (Y_test == 1))
TN = np.sum((y_pred == 0) & (Y_test == 0))
FP = np.sum((y_pred == 1) & (Y_test == 0))
FN = np.sum((y_pred == 0) & (Y_test == 1))

accuracy = (TP + TN) / len(Y_test)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2*(precision*recall)/(precision+recall)

print(f"\nĐộ chính xác (Accuracy): {accuracy:.4f}")
print(f"Độ chính xác dương (Precision): {precision:.4f}")
print(f"Độ nhạy (Recall): {recall:.4f}")
print(f"F1(B=1) score = {f1}")
print(f"Time: {-start_log+end_log}")

Các hệ số của mô hình (w):
[ -0.56656923   0.8609371    0.80833802  -0.5727334    0.15490466
   0.08641403   0.37425023  63.86299494  -2.08093648   1.30233708
  -1.18065667   0.29458534  -1.40451428   1.17974318   1.60598918
 -21.85639303   1.03961411  -0.46623334  -0.50144342   0.11174559
   0.21033231  -0.37179718  -0.95858394  -0.55374778  -1.20354252
   0.64654851   1.05381195   0.8820719    0.28512831   1.11706335
  -1.78375321  -0.96735713  -0.66337457   0.3485401    0.190321
   2.11044818   1.7534363    1.40249241  -1.44580391  -1.42462164]

Độ chính xác (Accuracy): 0.8934
Độ chính xác dương (Precision): 0.6522
Độ nhạy (Recall): 0.0169
F1(B=1) score = 0.03303964757709251
Time: 0.002248525619506836


In [588]:
def gaussian_prob(x, mean, std):
    std = max(std, 1e-6)
    return (1 / (sqrt(2*pi) * std)) * exp(- ((x - mean)**2) / (2*std**2))

def train_gnb(X, y):
    classes = np.unique(y)
    mu_list = []
    std_list = []
    pi_list = []
    for c in classes:
        X_c = X[y==c]
        mu_list.append(X_c.mean(axis=0))
        std_list.append(X_c.std(axis=0))
        pi_list.append(len(X_c)/len(X))
    return np.array(mu_list), np.array(std_list), np.array(pi_list)

def predict_gnb(X, mu_list, std_list, pi_list):
    y_pred = []
    for x in X:
        scores = []
        for mu, std, pi_c in zip(mu_list, std_list, pi_list):
            likelihood = np.prod([gaussian_prob(xi, mui, si) for xi, mui, si in zip(x, mu, std)])
            scores.append(likelihood * pi_c)
        y_pred.append(np.argmax(scores))
    return np.array(y_pred)

In [589]:
start_logNB = time.time()
mu_list, std_list, pi_list = train_gnb(X_train, Y_train)
y_pred_nb = predict_gnb(X_test, mu_list, std_list, pi_list)
end_logNB = time.time()

In [590]:
def evaluate(y_true, y_pred):
    TP = np.sum((y_true==1) & (y_pred==1))
    TN = np.sum((y_true==0) & (y_pred==0))
    FP = np.sum((y_true==0) & (y_pred==1))
    FN = np.sum((y_true==1) & (y_pred==0))

    accuracy = (TP + TN)/len(y_true)
    precision = TP/(TP+FP) if (TP+FP)>0 else 0
    recall = TP/(TP+FN) if (TP+FN)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return accuracy, precision, recall, f1, (TP, TN, FP, FN)

In [591]:
cc_nb, pre_nb, rec_nb, f1_nb, conf_nb = evaluate(Y_test, y_pred_nb)
print("\n=== Gaussian Naive Bayes ===")
print(f"Accuracy: {acc_nb:.4f}, Precision: {pre_nb:.4f}, Recall: {rec_nb:.4f}, F1: {f1_nb:.4f}")
print(f"Time: {end_logNB - start_logNB}")


=== Gaussian Naive Bayes ===
Accuracy: 0.8700, Precision: 0.2217, Recall: 0.8847, F1: 0.3545
Time: 0.8575026988983154


# Bài 3

In [ ]:
import pandas as pd

In [508]:
data = pd.read_csv("framingham.csv")
data = data.dropna(how="any", axis = 0)

In [509]:
def sigmoid(s):
    s = np.clip(s, -500, 500)
    return 1 / (1 + np.exp(-s))
def logistic_sigmoid_regression(X, y, w_init, eta, tol=1e-4, max_count=10000):
    w = [w_init]
    it = 0
    N = X.shape[1]
    d = X.shape[0]
    count = 0
    check_w_after = 20
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = y[i]
            zi = sigmoid(np.dot(w[-1].T, xi))
            w_new = w[-1] + eta * (yi - zi) * xi
            count += 1
            if count % check_w_after == 0:
                if np.linalg.norm(w_new - w[-check_w_after]) < tol:
                    return w
            w.append(w_new)
    return w

In [510]:
X = data.drop('TenYearCHD', axis=1).values.T
y = data['TenYearCHD'].values
X = np.vstack([np.ones((1, X.shape[1])), X]).T
print(X)


[[  1.     1.    39.   ...  26.97  80.    77.  ]
 [  1.     0.    46.   ...  28.73  95.    76.  ]
 [  1.     1.    48.   ...  25.34  75.    70.  ]
 ...
 [  1.     1.    50.   ...  25.97  66.    86.  ]
 [  1.     1.    51.   ...  19.71  65.    68.  ]
 [  1.     0.    52.   ...  21.47  80.   107.  ]]


In [511]:
train_size = int(0.7 * len(X))
X_train = X[:train_size,:]
Y_train =y[:train_size]
X_test = X[train_size:,:]
Y_test = y[train_size:]
X_mean = X_train[:,1:].mean(axis=0)
X_std = X_train[:,1:].std(axis=0)
X_train[:,1:] = (X_train[:,1:] - X_mean)/X_std
X_test[:,1:] = (X_test[:,1:] - X_mean)/X_std

In [512]:
d = X_train.shape[1]
w_init = np.random.randn(d, 1)
eta = 0.01
w = logistic_sigmoid_regression(X_train.T, Y_train, w_init, eta)
print(w[-1])

[[-2.00344629]
 [ 0.18568988]
 [ 0.56995806]
 [-0.07460883]
 [ 0.01385394]
 [ 0.22598039]
 [-0.01642637]
 [-0.03987643]
 [ 0.06879666]
 [-0.01064966]
 [ 0.19359789]
 [ 0.33345862]
 [-0.13788236]
 [ 0.06065003]
 [-0.06920722]
 [ 0.20566252]]


In [519]:
start_log = time.time()
y_prod = sigmoid(np.dot(w[-1].T, X_test.T))
y_pred = (y_prod > 0.5).astype(int)
end_log = time.time()
print("Các hệ số của mô hình (w):")
print(w[-1].flatten())
TP = np.sum((y_pred == 1) & (Y_test == 1))
TN = np.sum((y_pred == 0) & (Y_test == 0))
FP = np.sum((y_pred == 1) & (Y_test == 0))
FN = np.sum((y_pred == 0) & (Y_test == 1))

accuracy = (TP + TN) / len(Y_test)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2*(precision*recall)/(precision+recall)

print(f"\nĐộ chính xác (Accuracy): {accuracy:.4f}")
print(f"Độ chính xác dương (Precision): {precision:.4f}")
print(f"Độ nhạy (Recall): {recall:.4f}")
print(f"F1(B=1) score = {f1}")
print(f"Time: {-start_log+end_log}")

Các hệ số của mô hình (w):
[-2.00344629  0.18568988  0.56995806 -0.07460883  0.01385394  0.22598039
 -0.01642637 -0.03987643  0.06879666 -0.01064966  0.19359789  0.33345862
 -0.13788236  0.06065003 -0.06920722  0.20566252]

Độ chính xác (Accuracy): 0.8505
Độ chính xác dương (Precision): 0.6250
Độ nhạy (Recall): 0.0301
F1(B=1) score = 0.057471264367816084
Time: 0.00078582763671875
